In [2]:
from charDataset import *
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

train = torch.load('train_data.tx')
vocab = torch.load('vocab.tx')
n_words = len(vocab)


#c = CharDataset('train_data.tx','vocab.tx',50)

#retourne une sequence dans [0] et le reste dans [1]
def seq(text,debut,fin):
    return text[debut:fin], text[fin:]

# retourne une sequence codé dans [0] et le reste dans [1]
def code_seq(text,debut,fin,vocab):
    s = seq(text,debut,fin)
    return char2code(s[0],vocab), char2code(s[1],vocab)

text = 'je suis la'


def transform_one_hot(digit,n):
    y_onehot = torch.FloatTensor(n)
    y_onehot.zero_()
    y_onehot[digit] = 1
    return y_onehot


def transform_one_hot_sequence(sequence,n):
    seq_onehot = torch.FloatTensor(len(sequence),n)
    for i in range(len(sequence)):
        seq_onehot[i] = transform_one_hot(sequence[i],n)
    return seq_onehot

#transform_one_hot_sequence(char2code('n vreb br',vocab),n_words)

Je n'ai pas réussi a codé le RNN. Dans le forward, j'obtiens une somme des proba différentes de 1 mais je n'arrive pas à comprendre pourquoi 

In [62]:
class Modele(nn.Module):
    def __init__(self,l):
        super(Modele,self).__init__()
        self.l = l
        
        #init h a 0 de dimension d a changer a chaque linear donc 2 param a apprendre w_h et w_x
        # Il faut 3 linear: 
        # - un pour passer de Rv -> Rd
        # le 1er converti une lettre (dimension v) dans l'espace latent
        # - un pour passer de Rd -> Rd
        # le 2e predit la suite dans l'espace latent
        # - un pour passer de Rd -> Rv
        # le 3e transforme dans v
        # on doit faire un logSoftmax pour avoir p(y1 | x_0,h_0) puis tirage puis one_hot
        self.encodage = nn.Linear(n_words,self.l, bias=False)
        self.decodage = nn.Linear(self.l,self.l, bias=False)
        self.d2 = nn.Linear(self.l, n_words, bias=False)
        self.tanh = nn.Tanh()
        
    def forward_train(self,x,y, stopWord = "~", test= False, size_max = 140):
        #mettre formule du forward 
        # ne marche pas au niveau du decodage je sais pas pourquoi
        H = [Variable(torch.zeros(x.size(0),self.l))]
        logSoftmax = nn.LogSoftmax()
        _, argmax = x[:,0].max(dim=-1)
        predictions = [argmax]
        probPrediction = []
        size = 1
        out = [x]
        size_max = size_max if test else len(y)
        while(size<size_max and predictions[-1].data[0] != stopWord):
            h = self.tanh(self.encodage(x[-1]) + self.decodage(H[-1]))
            H.append(h)
            #print('somme : ' , torch.sum(h))
            pred = self.d2(h)
            probPrediction.append(logSoftmax(pred))
            # somme non egale à 1
            # un peu étrange mais je ne comprend pas pourquoi
            #print('somme : ' , torch.sum(logSoftmax(predProba)))
            if test : 
                out.append(probPrediction)
                predictions.append(self.tirage(probPrediction))
            else:
                out.append(y)
            size += 1
#         print(probPrediction)
#         print(predictions)
        return torch.cat(probPrediction),torch.cat(predictions)
    
    def tirage(self, distribution):
        distrib = torch.cat([r / r.sum(dim=-1) for r in distribution])
        return distrib.multinomial(1)

    def predict(self,x):
        _,decoded = self.forward(x).max(dim=1)
        return decoded

In [66]:
    
m = Modele(2)
text = 'je suis la'
test = code_seq(text,0,3,vocab)
x_train = Variable(transform_one_hot_sequence(test[0],n_words))
y_train = Variable(transform_one_hot_sequence(test[1],n_words))

a,b = m.forward_train(x_train,y_train)
print(type(a))

<type 'tuple'>
